# Further discussion on breakpoint detection
In this notebook, we explore in more detail the issue of detecting structural breaks in the regression of log(vacancy rate) on the log(unemployment rate).

These breakpoints are needed because the Beveridge Elasticity is computed directly from this regression. 

## Get our economic data
For computing the BUG, we need:
  * unemployment rate: u
  * vacancy rate: v
  * beveridge curve elasticity (computed from u, v, and breakpoints on the v/u series)
  * social value of non-work (default is zeta = 0.26)
  * recruting costs (default is kappa = 0.92)
  
For context in the plots, we also want recession information.
<br>

### Data source: 

![image.png](https://fred.stlouisfed.org/images/fred-logo-2x.png)
<br>

The St.Louis Fed has an [API](https://fred.stlouisfed.org/docs/api/fred/series_observations.html) which alows you to pull data programatically.

**You will need a registered API key**. See [here](https://fred.stlouisfed.org/docs/api/api_key.html) for info.

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import json, timeit
import ruptures as rpt
from kneed import KneeLocator

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('fivethirtyeight')

In [ ]:
import sys
sys.path.insert(0, '../')
import bug

In [ ]:
# a helper function for pulling json formatted data from FRED
def get_series(series_id, my_key, start_date='1951-01-01'):
    
    link = 'https://api.stlouisfed.org/fred/series/observations?series_id='+series_id+'&observation_start='+start_date+'&file_type=json&api_key='+my_key
    content = urllib.request.urlopen(link).read()
    data = json.loads(content)
    df = pd.DataFrame(data['observations'])[['date', 'value']]
    
    return df

In [ ]:
## This key is an EXAMPLE ONLY!! 
# You will need to replace this with your actual registered key for the FRED API
my_key = 'abcdef1234567890abcdef'
my_key = 'ad6e20b7a328d09ddd3917b5f43ce3fa'

In [ ]:
# Recession information
recession = get_series('USREC', my_key)
recession['date'] = pd.to_datetime(recession['date'] )
recession['value'] = recession['value'].astype(int)
recession.set_index('date', inplace=True)
recession['starts'] = (recession.value- recession.value.shift(1) ==1)
recession['ends'] = (recession.value- recession.value.shift(1) ==-1)
starts = recession.index[recession['starts']==1].to_list()
ends = recession.index[recession['ends']==1].to_list()

In [ ]:
# unemployment rate
unempl = get_series('UNRATE', my_key)
unempl['date'] = pd.PeriodIndex(pd.to_datetime(unempl['date'] ).dt.to_period('m') )
u = pd.Series(data=unempl['value'].values,index=unempl['date'], name='unempl_rate')
u = pd.to_numeric(u)/100.0
u_q = u.resample('Q').mean()

In [ ]:
# vacancy info
hwi = pd.read_csv("../new_data/HWI_index.txt", skiprows=6, header=None,delim_whitespace=True)
hwi['date'] = pd.PeriodIndex(pd.to_datetime(hwi[0].str[:4]+'-'+hwi[0].str[-2:] ).dt.to_period('m') )
vac_proxy = pd.Series(data=pd.to_numeric(hwi[1].values),index=hwi['date'], name='help-wanted index') 

In [ ]:
labor_lev = get_series('CLF16OV', my_key)
labor_lev['date'] = pd.PeriodIndex(pd.to_datetime(labor_lev['date'] ).dt.to_period('m') )
lfl = pd.Series(data=pd.to_numeric(labor_lev['value'].values),index=labor_lev['date'], name='labor_force_level')

In [ ]:
nf_vac = get_series('JTSJOL', my_key)
nf_vac['date'] = pd.PeriodIndex(pd.to_datetime(nf_vac['date'] ).dt.to_period('m') )
vac = pd.Series(data=pd.to_numeric(nf_vac['value'].values),index=nf_vac['date'], name='nonfarm_vacancies')
vac_rate = vac/lfl
v = pd.concat([vac_proxy.loc[:'2000-12']/100., vac_rate.loc['2001-01':]],)
v_q = v.resample('Q').mean()

### Beveridge Curve

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1, 1, 1)
ax.plot(np.log(u), np.log(v), linewidth=1, color='darkblue')

bug.format_plot(ax, xgrid=False )

plt.ylabel('Log Vacancy Rate', fontsize=12)
plt.xlabel('Log Unemployment Rate', fontsize=12)
plt.title('Beveridge Curve (monthly)', fontsize=14)

In [ ]:
log_u_q = np.log(u.resample('Q').mean())
log_v_q = np.log(v.resample('Q').mean())

last_index = min(log_u_q.last_valid_index(), log_v_q.last_valid_index())
log_u_q = log_u_q.loc[:last_index]
log_v_q = log_v_q.loc[:last_index]

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1, 1, 1)
ax.plot(log_u_q, log_v_q, linewidth=1,color='darkblue')

bug.format_plot(ax, xgrid=False)

plt.ylabel('Log Vacancy Rate', fontsize=12)
plt.xlabel('Log Unemployment Rate', fontsize=12)
plt.title('Beveridge Curve (quarterly)', fontsize=15)

## ruptures package
We are leveraging the [ruptures](https://centre-borelli.github.io/ruptures-docs/) package in python to perform the breakpoint detection. The package implements most of the algorithms surveyed in the paper: C.Truong, L.Oudre, N.Vayatis. (2020) "Selective review of offline change point detection methods." *Signal Processing*, 167:107299. [pdf](http://www.laurentoudre.fr/publis/TOG-SP-19.pdf)

The design of the rutures package allows the user to seperatly specify the [search method](https://centre-borelli.github.io/ruptures-docs/user-guide/detection/dynp/) and the [cost function](https://centre-borelli.github.io/ruptures-docs/user-guide/costs/costl1/); thus, allowing for modular composition of breakpoint detection approaches. Different compositions will correspond to different algorithms in the literature.


For example, using `rpt.Dynp` for the dynamic programing search and `rpt.costs.CostLinear` as the cost function, we get an implementation of the Bai-Perron (2003) algorithm that is used in the Michaillat & Saez paper. ([M&S GitHub link](https://github.com/pascalmichaillat/unemployment-gap))

### Search methods
The ruptures package implements 6 different search methods for finding breakpoints.
  * `Dynp`: "It finds the (exact) minimum of the sum of costs by computing the cost of all subsequences of a given signal. It is called "dynamic programming" because the search over all possible segmentations is ordered using a dynamic programming approach." This is the method used in Bai-Perron. *Works with pre-specified number of break-points.* 
  * `Pelt`: This algorithm relies on a pruning rule to find an approximate solution when the enumeration of all possible partitions is impractical or impossible. *Works when number of breakpoints is unknown.* (Uses a penatly parameter to select number of breakpoints, rather than specifying the number directly.)
  * `KernelCPD`: A faster C implementation that can perform both dynamic programming and PELT.
  * `Bingseg`: A very simple, greedy binary segmentation approach. Advantages include very low complexity and can be used when number of breakpoints/sub-series regimes is known or unknown.
  * `BottomUp`: Another simple approach like binary segmentation, with similar advantages (Low complexity and can work with known or unknown number of breakpoints). But this one is generous, not greedy.
  * `Window`: Another simple approach based on sliding windows. As with bottom-up or binary segnmentaton, it has low complexity and can work with known or unknown number of breakpoints. Window width parameter needs to be smaller than the minimum sub-series length, so may not work for detecting short sub-series. 
  
### Let's try...
Our series are not too long (857 for monthly or 286 for quarterly), so computational complexity is not much of a concern. We discard the choices of `Pelt` and `KernelCPD` for this reason. We also remove the `Window` method from contention because the small window size needed to detect the 8 quarters post-COVID regime is likely too small to give robust breakpoint estimates overall.


So we will try the simple methods, `Binseg` and `BottomUp`, and compare them to the `Dynp` method (Dynp is equivalent to the method used in B-P).

### Cost functions
Besides `CostLinear`, there are 9 other cost functions available in ruptures, as well as a method for declaring a custom cost. (See [here](https://centre-borelli.github.io/ruptures-docs/user-guide/costs/costcustom/).) Should we try any of these? 

*No.*

Actually, in this application, `CostLinear` appears most appropriate. Several of the other cost functions operate on univariate series only, so they do not incorporate covariates, which we need in this case.

In this application, *y=log(vacanacy)*, and *x=log(unemployment)*, and we find the breakpoints such that the sum of the squared residuals across peice-wise OLS regressions is minimized. Not only does that give us our breakpoints, but this also immediately gives us the quantity we desire, the Beverdige elasticity, as elasticity is the simply the negative of the OLS estimated coeff for log(unemployment).

Therefore linear cost seems the most appropriate for this application.

## Experiments with breakpoint detection
### Fixing number fo breakpoints

Recall the breakpoints from the M&S paper were [0, 41, 84, 153, 194, 235, 276].

Note that was 5 breakpoints for the period 1951--2019, whereas we now are looking at 6 breakpoints for 1951--current year.

#### Timing tests
To run timing tests, uncomment the lines with the %timeit magic

In [ ]:
n_bkps = 6
min_size = 7
model ='linear'

In [ ]:
# put the data in the format that ruptures package likes
y = np.array(log_v_q)
X = np.vstack((np.array(log_u_q), np.ones(len(y)))).T
signal = np.column_stack((y.reshape(-1, 1), X))

## Dynamic programming

In [ ]:
fit_dynp = rpt.Dynp(model=model, min_size=min_size, jump=1).fit(signal)
%timeit fit_dynp = rpt.Dynp(model=model, min_size=min_size, jump=1).fit(signal) 

In [ ]:
est_dynp = fit_dynp.predict(n_bkps=n_bkps)
%timeit est_dynp = fit_dynp.predict(n_bkps=n_bkps)

In [ ]:
est_dynp.insert(0,0)
est_dynp

### Plot of Bev Elasticity calculated from Dynamic Programming
#### (This is the M&S Paper method)

In [ ]:
e_dynp, _ = bug.compute_beveridge_elasticity(u_q, v_q, bkps_in=est_dynp)

bug.plot_beveridge_elasticity_series(e_dynp, recession_dates=[starts, ends], draw_legend=True)
plt.ylim(0,1.25)

## Binary Segmentation
Specified number of breakpoints

In [ ]:
fit_bin = rpt.Binseg(model=model, min_size=min_size, jump=1).fit(signal)
%timeit fit_bin = rpt.Binseg(model=model, min_size=min_size, jump=1).fit(signal)

In [ ]:
est_bin = fit_bin.predict(n_bkps=n_bkps)
%timeit est_bin = fit_bin.predict(n_bkps=n_bkps)

In [ ]:
est_bin.insert(0,0)
est_bin

### Plot of Bev Elasticity calculated from Binary Segmentation

In [ ]:
e_bin, _ = bug.compute_beveridge_elasticity(u_q, v_q, bkps_in=est_bin)
bug.plot_beveridge_elasticity_series(e_bin, recession_dates=[starts, ends], draw_legend=True)
plt.ylim(0,1.25)

### Bottom-Up
Specified number of breakpoints

In [ ]:
fit_bup = rpt.BottomUp(model=model, min_size=min_size, jump=1).fit(signal)
%timeit fit_bup = rpt.BottomUp(model=model, min_size=min_size, jump=1).fit(signal)

In [ ]:
est_bup = fit_bup.predict(n_bkps=n_bkps)
%timeit est_bup = fit_bup.predict(n_bkps=n_bkps)

In [ ]:
est_bup.insert(0,0)
est_bup

In [ ]:
e_bup, _ = bug.compute_beveridge_elasticity(u_q, v_q, bkps_in=est_bup)
bug.plot_beveridge_elasticity_series(e_bup, recession_dates=[starts, ends], draw_legend=True)
plt.ylim(0,1.25)

In [ ]:
summary_6_breaks = pd.DataFrame({'Paper':[0, 41, 84, 153, 194, 235, 275, 'NA'], 
                                 'DynProg':est_dynp, 'BottomUp':est_bup, 
                                 'BinarySeg':est_bin, })

In [ ]:
summary_6_breaks

## The resulting estimated BUG for different methods

In [ ]:
e_ref, _ = bug.compute_beveridge_elasticity(u_q, v_q, 
                                         bkps_in=np.array([0, 41, 84, 153, 194, 235, 275, 286]) )
gap_ref = bug.compute_unemployment_gap(u_q, v_q, e_ref['E'])
gap_dynp = bug.compute_unemployment_gap(u_q, v_q, e_dynp['E'])
gap_bin = bug.compute_unemployment_gap(u_q, v_q, e_bin['E'])
gap_bup = bug.compute_unemployment_gap(u_q, v_q, e_bup['E'])

In [ ]:
ax = gap_dynp.plot( linewidth=3, figsize=(9, 6), label='Dynamic Prog')
gap_bin.plot(ax=ax, linewidth=2, label='Binary Seg')
gap_bup.plot(ax=ax, linewidth=2, color='k',linestyle='dashed',label='Bottom Up')
plt.axhline(y=0, color='magenta', linewidth=2,)

for b in summary_6_breaks.DynProg[1:-1]:
    plt.axvline(x=u_q.index[b], color='blue', linewidth=1)

for b in summary_6_breaks.BinarySeg[1:-1]:
    plt.axvline(x=u_q.index[b], color='red', linewidth=1)
    
for b in summary_6_breaks.BottomUp[1:-1]:
    plt.axvline(x=u_q.index[b], color='black', linewidth=1)
    
plt.legend()
plt.ylim(-.02,.1)
bug.format_plot(ax, [starts,ends])
plt.ylabel('Unemployment Gap', fontsize=12)
plt.title('Unemployment Gap', fontsize=14)

### Discussion
We see the resulting breakpoints between the three methods are quite similar, especially for the period after 1972. This leads to the estimated BUG being similar as well.

There are more differences in the earlier part of the series.

But since the dynamic programming method finds an *exact* solution to minimizing the sum of costs (recall cost is SSR in this case), the main reason to favor a different algorithm like Binary Segmentation or Bottom-up would be computational. In our case, we have short series, and we actually found the Dynamic Porgramming method to be the **FASTEST!**

## Number of breakpoints
The experiments above with search method assumed the correct number of breakpoints was known. (We assumed 6 in this case.)

Let's investigate.

The Binseg or BottomUp search methods can be used with a penalty parameter to determine the number of breakpoints. But that still relies on some rules-of-thumb for setting that penalty.

And from above, we have seen the Dynamic Porgramming approach is the fastest anyway. 

So, we can just cycle through a number of breakpoints, and see what it looks like:

## Different num breakpoints w/ Dynamic Prog

In [ ]:
fit_dynp = rpt.Dynp(model='linear', min_size=10, jump=1).fit(signal)
_ = fit_dynp.predict(n_bkps=10)
num_breaks = [8,6,4,2]
for b in num_breaks:
    est_dynp = fit_dynp.predict(n_bkps=b)
    est_dynp.insert(0,0)
    e_dynp, _ = bug.compute_beveridge_elasticity(u_q, v_q, bkps_in=est_dynp)
    bug.plot_beveridge_elasticity_series(e_dynp, 
                                         recession_dates=[starts, ends], 
                                         draw_legend=True, legend_loc=1)  
    plt.ylim(0,1.5)
    plt.title('num breaks: '+str(len(est_dynp)-2), fontsize=14)

### How to decide the number of breaks
Other than just eyeballing?

Bai & Perron (2003) suggest supF type tests; e.g. for no break (*m=0*) vs *m=k* breaks. Further, this can be extended to test of *m* breaks vs *m+1* breaks. The possibility of examining BIC or other information criteria is also discussed, but B-P warn this sometimes fails under conditions of serial dependence in the errors. 

The ruptures package does not include hypothesis testing for determining the number of breaks, so we'll have to write some evaluation tools for ourselves. 

In [ ]:
# put the data in the format that ruptures package likes
y = np.array(log_v_q)
X = np.vstack((np.array(log_u_q), np.ones(len(y)))).T
signal = np.column_stack((y.reshape(-1, 1), X))

In [ ]:
n_bkps_max =12

In [ ]:
Eval = bug.evaluate_num_breaks(signal, n_bkps_max, min_size=9)

In [ ]:
k = KneeLocator(np.arange(0,n_bkps_max + 1), Eval.ssr, curve="convex", direction="decreasing")
ax = k.plot_knee()
plt.title('SSR')
plt.xlabel("Number of change points")

In [ ]:
k = KneeLocator(np.arange(0,n_bkps_max + 1), Eval.bic, curve="convex", direction="decreasing")
ax = k.plot_knee()
plt.title('BIC')
plt.xlabel("Number of change points")

In [ ]:
k = KneeLocator(np.arange(0,n_bkps_max + 1), Eval.lwz, curve="convex", direction="decreasing")
ax = k.plot_knee()
plt.title('LWZ Modified BIC')
plt.xlabel("Number of change points")

In [ ]:
# using the knee found on LWZ
chosen_k = k.knee
opt = Eval.bkps[chosen_k]
opt

In [ ]:
e_opt, coeffs_opt = bug.compute_beveridge_elasticity(log_u_q, log_v_q, bkps_in=opt )
gap_opt = bug.compute_unemployment_gap(u_q, v_q, e_opt['E'])

In [ ]:
in_bks = [u_q.index[b] for b in opt[:-1]]
in_bks

In [ ]:
ax = bug.plot_beveridge_gap_series(gap_opt, in_bks, recession_dates=[starts, ends], )

plt.ylim(-.02,.1)

In [ ]:
bug.plot_beveridge_elasticity_series(e_opt, recession_dates=[starts, ends], draw_legend=True)
plt.ylim(-.04,2.2)

In [ ]:
bug.plot_beveridge_curve_segments(log_u_q, log_v_q, opt)

In [ ]:
bug.plot_beveridge_curve_fits(log_u_q, log_v_q, opt, coeffs_opt)